In [2]:
import pandas as pd
import numpy as np
import torch
import os
import sys
sys.path.append("/home/docker_current/py_files/MLCLIP_exp")
from MLCLIP_utils import get_text_encode_model, get_image_encode_model
from metaclip_arch.metaclip_v1 import MetaCLIP

device = "cuda:1" if torch.cuda.is_available() else "cpu"

metaclip_model = MetaCLIP()
path_weights = "metaclip_v1.pth"

metaclip_model.load_state_dict(torch.load(path_weights, map_location=torch.device('cpu')) )
metaclip_model.to(device)

MetaCLIP(
  (encode_image): ImageEncoder(
    (fc1): Linear(in_features=768, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
  )
  (encode_text): TextEncoder(
    (fc1): Linear(in_features=768, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
  )
)

In [4]:
from sklearn.model_selection import train_test_split

with open('/home/docker_current/py_files/MLCLIP_exp/image_feat_array.npy', 'rb') as f:
    image_feat_array = np.load(f)

with open('/home/docker_current/py_files/MLCLIP_exp/text_feat_array.npy', 'rb') as f:
    text_feat_array = np.load(f)

X_train, X_test, y_train, y_test = train_test_split(text_feat_array, image_feat_array, 
                                                        test_size=0.25, random_state=42)

In [5]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

def get_similarity(image_emb, text_emb):
    sim =cos(torch.Tensor(image_emb), 
                        torch.Tensor(text_emb))
    return sim

In [30]:
ind = 0
sample_image_feat = X_test[ind].reshape(1, -1)
sample_text_feat = y_test[ind].reshape(1, -1)

get_similarity(sample_image_feat, sample_text_feat)

tensor([0.2866])

In [31]:
metaclip_sample_image_feat = torch.Tensor(sample_image_feat).to(device)
metaclip_sample_text_feat = torch.Tensor(sample_text_feat).to(device)

metaclip_sample_image_feat = metaclip_model.encode_image(metaclip_sample_image_feat).cpu()
metaclip_sample_text_feat = metaclip_model.encode_text(metaclip_sample_text_feat).cpu()

get_similarity(metaclip_sample_image_feat, metaclip_sample_text_feat)

tensor([0.3773], grad_fn=<DivBackward0>)